# CoastWatch


In [1]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime, timezone, timedelta
from Elves import Download, Image_Processing, Shoreline, Toolbox, Transects, VegetationLine
import mpl_toolkits as mpl
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
from sklearn.datasets import load_diabetes
import seaborn as sns; sns.set()
import math
import geemap
import ee
import pprint
from shapely import geometry
from shapely.geometry import Point, LineString
import geopandas as gpd
import matplotlib.cm as cm
import pyproj
from IPython.display import clear_output
import scipy
from scipy import optimize
import csv
import math

ee.Initialize()

## Region of Interest Selection

Two options for image retrieval: 1st involes selecting region for the map generated below; 2nd involes inputting lat,lon coords manually.

**Option 1**

In [5]:
"""
Run this cell to generate a map. Use the polygon drawing tool on the left-hand side to 
draw out the region of coast you're interested in.
"""

Map = geemap.Map(center=[0,0],zoom=2)
Map.add_basemap('HYBRID')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Toggl…

In [6]:
roi = Map.user_roi.geometries().getInfo()[0]['coordinates']
polygon = [[roi[0][0],roi[0][3],roi[0][1],roi[0][2]]]
point = ee.Geometry.Point(roi[0][0])

**Option 2**

In [2]:
##Below are example coordinates

##ST ANDREWS
lonmin, lonmax = -2.842023, -2.774955
latmin, latmax = 56.338343, 56.368490

##FELIXSTOWE
#lonmin, lonmax = 1.316128, 1.370888
#latmin, latmax = 51.930771, 51.965265

##BAY OF SKAILL
#lonmin, lonmax = -3.351555, -3.332693
#latmin, latmax = 59.048456, 59.057759

##SHINGLE STREET
#lonmin, lonmax = 1.446131, 1.460008
#latmin, latmax = 52.027039, 52.037448


point = ee.Geometry.Point([lonmin, latmin]) 
polygon = [[[lonmin, latmin],[lonmax, latmin],[lonmin, latmax],[lonmax, latmax]]]

## Image Retrieval

In [3]:
# it's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
polygon = Toolbox.smallest_rectangle(polygon)

# date range
dates = ['2021-05-01', '2021-07-02']
# satellite missions
# Input a list of containing any/all of 'L5', 'L8', 'S2'
sat_list = ['S2']

projection_epsg = 27700
image_epsg = 32630

# name of the site
sitename = 'Test'
# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'Data')

# put all the inputs into a dictionnary
inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath}

direc = os.path.join(filepath, sitename)

if os.path.isdir(direc) is False:
    os.mkdir(direc)

# before downloading the images, check how many images are available for your inputs
Download.check_images_available(inputs);

Images available between 2021-05-01 and 2021-07-02:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 20 images
  Total: 20 images


In [4]:
Sat = Toolbox.image_retrieval(point,dates, sat_list)
metadata = Toolbox.metadata_collection(sat_list, Sat, filepath, sitename)

Metadata already exists and was loaded


The next 2 sections are for extracting the shoreline and vegetation edge respectively. (No need to do shoreline before veg edge)

## Shoreline Extraction

In [5]:
BasePath = 'Data/' + sitename + '/Shorelines'

if os.path.isdir(BasePath) is False:
    os.mkdir(BasePath)

settings = {
    # general parameters:
    'cloud_thresh': 0.4,        # threshold on maximum cloud cover
    'output_epsg': projection_epsg,#metadata[sat_list[0]]['epsg'][0], # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': False,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 50,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) for buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 100,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'dark',   # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    # add the inputs defined previously
    'inputs': inputs,
    'hausdorff_threshold':3500
}

Digitize a reference shoreline

In [5]:
%matplotlib qt
settings['reference_shoreline'] = Image_Processing.get_reference_sl(metadata, settings, polygon, dates)
settings['max_dist_ref'] = 100 # max distance (in meters) allowed from the reference shoreline

NameError: name 'settings' is not defined

Batch shoreline detection

In [8]:
%matplotlib qt
output, output_latlon, output_proj = Shoreline.extract_shorelines(metadata, settings, polygon, dates)

Mapping shorelines:
L5:   0%

KeyError: 'projection_epsg'

In [5]:
#Alternatively, load the output file if previously processed
filepath = os.path.join(inputs['filepath'], sitename)
with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'rb') as f:
    output = pickle.load(f)
with open(os.path.join(filepath, sitename + '_output_latlon' + '.pkl'), 'rb') as f:
    output_latlon = pickle.load(f) 
with open(os.path.join(filepath, sitename + '_output_proj' + '.pkl'), 'rb') as f:
    output_proj = pickle.load(f) 

Then remove duplicates and images with inaccurate georeferencing (threhsold at 10m)

In [ ]:
output = Toolbox.remove_duplicates(output) # removes duplicates (images taken on the same date by the same satellite)
output_latlon = Toolbox.remove_duplicates(output_latlon)
output_proj = Toolbox.remove_duplicates(output_proj)

In [ ]:
#FN THAT REMOVES BY ACTIVE CHOICE GOES HERE

For use in GIS applications, you can save the mapped shorelines as a GEOJSON layer which can be easily imported into QGIS for example. You can choose to save the shorelines as a collection of lines or points (sometimes the lines are crossing over so better to use points).

In [28]:
##Visual Display of Shorelines (will tank PC if you've produced too many lines)

#Creates map object centred at ROI + adds compiled satellite image as base-layer
Map = geemap.Map(center=[polygon[0][0][1],polygon[0][0][0]],zoom=12)
Map.add_basemap('HYBRID')

#Generates colours for lines to be drawn in. Check out https://seaborn.pydata.org/tutorial/color_palettes.html for colour options...
palette = sns.color_palette("bright", len(output['shorelines']))
palette = palette.as_hex()

#Choose 'points' or 'lines' for the layer geometry
geomtype = 'points'

for i in range(len(output['shorelines'])-82):    
    shore = dict([])
    shore = {'dates':[output_latlon['dates'][i]], 'shorelines':[output_latlon['shorelines'][i]], 'filename':[output_latlon['filename'][i]], 'cloud_cover':[output_latlon['cloud_cover'][i]], 'geoaccuracy':[output_latlon['geoaccuracy'][i]], 'idx':[output_latlon['idx'][i]], 'Otsu_threshold':[output_latlon['Otsu_threshold'][i]], 'satname':[output_latlon['satname'][i]]}
    
    if len(shore['shorelines'][0])==0:
        continue
    
    gdf = Toolbox.output_to_gdf(shore, geomtype)
    Line = geemap.geopandas_to_ee(gdf, geodesic=True)

    Map.addLayer(Line,{'color': str(palette[i])},'coast'+str(i))

Map

Map(center=[55.523971, -4.674243], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [ ]:
#Saves the shorelines as individual shape-files locally under /Shorelines.

#Choose 'points' or 'lines' for the layer geometry
geomtype = 'lines'

for i in range(len(output['shorelines'])):
    filename = "Shorelines/" + sitename + '_' + str(output['dates'][i])
    
    shore = dict([])
    shore = {'dates':[output_latlon['dates'][i]], 'shorelines':[output_latlon['shorelines'][i]], 'filename':[output_latlon['filename'][i]], 'cloud_cover':[output_latlon['cloud_cover'][i]], 'geoaccuracy':[output_latlon['geoaccuracy'][i]], 'idx':[output_latlon['idx'][i]], 'MNDWI_threshold':[output_latlon['MNDWI_threshold'][i]], 'satname':[output_latlon['satname'][i]]}
    
    gdf = Toolbox.output_to_gdf(shore, geomtype)
    
    gdf.to_file(filename)

In [ ]:
#Saves the shorelines as individual shape-files locally under /Shorelines.
direc = os.path.join(filepath, '/Shorelines')
geomtype = 'lines'
name_prefix = 'Data/' + sitename + '/Shorelines'

if os.path.isdir(direc) is False:
    os.mkdir(direc)

Toolbox.save_shapefiles(output_latlon, geomtype, name_prefix, sitename)

ref_line = np.delete(settings['reference_shoreline'],2,1)
ref = {'dates':['3000-12-30'], 'shorelines':[ref_line], 'filename':[0], 'cloud_cover':[0], 'geoaccuracy':[0], 'idx':[0], 'Otsu_threshold':[0], 'satname':[0]}
    
Toolbox.save_shapefiles(ref, geomtype, name_prefix, sitename)

In [5]:
#Produces Transects and Coast shape-files for each processed shoreline

SmoothingWindowSize = 21
NoSmooths = 100
TransectSpacing = 10
DistanceInland = 350
DistanceOffshore = 350
BasePath = 'data/' + sitename + '/Shorelines'

Transects.produce_transects(SmoothingWindowSize, NoSmooths, TransectSpacing, DistanceInland, DistanceOffshore, image_epsg, sitename, BasePath + '/' + sitename + '_referenceLine')

#(Optional) Produces transects for all produced lines
#Transects.produce_transects_all(SmoothingWindowSize, NoSmooths, TransectSpacing, DistanceInland, DistanceOffshore, projection_epsg, BasePath)

DriverError: data/Troon/Veglines/Troon_referenceLine/Troon_referenceLine.shp: No such file or directory

In [ ]:
#Defines all transects in a library.

TransectSpec =  '/' + sitename + '_referenceLine/Transect.shp'
geo = gpd.read_file(BasePath+TransectSpec)

transect_latlon, transect_proj = Transects.stuffIntoLibrary(geo, image_epsg, projection_epsg, filepath, sitename)

In [6]:
#Or just load them if already produced
with open(os.path.join(filepath, sitename + '_transect_proj' + '.pkl'), 'rb') as f:
    transects_proj = pickle.load(f)
with open(os.path.join(filepath, sitename + '_transect_latlon' + '.pkl'), 'rb') as f:
    transects_latlon = pickle.load(f)

In [ ]:
# defines the along-shore distance over which to consider vegetation line points to compute the median intersection (robust to outliers)
settings['along_dist'] = 25
cross_distance = Transects.compute_intersection(output_proj, transects_proj, settings) 

Simple plot of the mapped shorelines. The coordinates are stored in the output dictionnary together with the exact dates in UTC time, the georeferencing accuracy and the cloud cover.

## Analysis - Shoreline

In [7]:
#Shorelines displayed in matplotlib

#fig = plt.figure(figsize=[15,8])
#plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
#plt.xlim(0,3*10**7)
#plt.ylim(0,0.7*10**9)
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.-', label=datetime.strptime(date,'%Y-%m-%d'))
plt.legend();
plt.show()

In [21]:
#WIP

fig, ax = plt.subplots(1, figsize=[15,8])
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
ax.set_xlim(146000, 153000)

#axins = zoomed_inset_axes(ax, zoom=6, loc=7)
mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")

for i in range(len(output['shorelines'])):
    if i%16==0:
        sl = output['shorelines'][i]
        date = output['dates'][i]
        ax.plot(sl[:,0], sl[:,1], '-', label=datetime.strptime(date,'%Y-%m-%d'))
        #axins.plot(sl[:,0], sl[:,1], '-')

ax.legend(loc=2)

#axins.set_xticklabels('')
#axins.set_yticklabels('')

In [18]:
fig = plt.figure(figsize=[15,8], tight_layout=True)
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output_proj['shorelines'][i]
    date = output_proj['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=datetime.strptime(date,'%Y-%m-%d').strftime('%d-%m-%Y'))
for i,key in enumerate(list(transects_proj.keys())):
    plt.plot(transects_proj[key][0,0],transects_proj[key][0,1], 'bo', ms=5)
    plt.plot(transects_proj[key][:,0],transects_proj[key][:,1],'k-',lw=1)
    #plt.text(transects_proj[key][0,0]-100, transects_proj[key][0,1]+100, key,va='center', ha='right', bbox=dict(boxstyle="square", ec='k',fc='w'))

Now, intersect the transects with the 2D shorelines to obtain time-series of cross-shore distance.

The time-series of shoreline change for each transect are saved in a .csv file in the data folder (all dates are in UTC time). 

In [26]:
# defines the along-shore distance over which to consider shoreline points to compute the median intersection (robust to outliers)
settings['along_dist'] = 25 
cross_distance = Transects.compute_intersection(output_proj, transects_proj, settings) 

Time-series of the shoreline change along the transects saved as:
C:\Users\Luke\OneDrive - Durham University\Research_Work\CoastWatch\Data\ML_Test6\transect_time_series.csv


Sorts Data Into Separate Years

Function which separates data into organised lists (organised first by years then by months). It also averages over these values (avg. for month x or year y).

Average Yearly Shoreline Over Time (All Transects Plotted Independently)

In [15]:
for i in range(len(cross_distance.keys())):
    KEY = 'Transect_'+str(i+1)
    a, b, c, d, e = Toolbox.Separate_TimeSeries_year(output_proj, KEY)
    plt.plot(d,e)

plt.ylabel("Average Yearly Vegetation Line Position")
plt.xlabel("Year")
plt.show()

NameError: name 'cross_distance' is not defined

Seasonal Variation (All Transects Plotted Independently)

In [ ]:
#plt.figure(figsize=[15,12])

months = ["Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", "Sept", "Oct", "Nov", "Dec"]
Month_dict = {"Jan":[], "Feb":[], "Mar":[], "Apr":[], "May":[], "June":[], "July":[], "Aug":[], "Sept":[], "Oct":[], "Nov":[], "Dec":[]}

Per_Transect_Monthly_DistanceAvg = []
test1 = []
test2 = []

for i in range(len(cross_distance.keys())):
    KEY = 'Transect_'+str(i+1)
    a, b, c, d, e = Toolbox.Separate_TimeSeries_month(output_proj,KEY)

    plt.scatter(months,e,label=KEY)
    test1.append(d)
    test2.append(e)
    
for k in range(len(test2)):
    for h in range(len(test2[k])):
        for i,j in enumerate(Month_dict):
            if (i+1) == test1[k][h]:
                avg[j].append(test2[k][h])
                
Average_Month = []

for i,j in enumerate(Month_dict):
    Average_Month.append(sum(avg[j])/len(avg[j]))
    
plt.plot(months,Average_Month)
    
#plt.legend()
plt.ylabel("Averaged Monthly Shoreline")
plt.show()

Coastal Variation (1984-2020) for each transect alongside probability dist. fn.

In [27]:
fig = plt.figure(figsize=[15,8], tight_layout=True)
gs = gridspec.GridSpec(len(cross_distance),2, wspace=0.035, width_ratios=[3,1])
gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.2)
for i,key in enumerate(cross_distance.keys()):
    if np.all(np.isnan(cross_distance[key])):
        continue
    ax = fig.add_subplot(gs[i,0])
    ax.grid(linestyle=':', color='0.5')
    ax.set_ylim([-100,110])
    ax.plot(output['dates'], cross_distance[key]- np.nanmedian(cross_distance[key]), '-o', ms=6, mfc='w')
    #ax.set_ylabel('distance [m]', fontsize=12)
    ax.text(0.5,0.95, key, bbox=dict(boxstyle="square", ec='k',fc='w'), ha='center',va='top', transform=ax.transAxes, fontsize=14)
    if i!= len(cross_distance.keys())-1:
        ax.set_xticklabels('')
    ax = fig.add_subplot(gs[i,1])
    #ax.set_xlim([-50,50])
    ax.set_xlim([0,0.015])
    sns.distplot(cross_distance[key]- np.nanmedian(cross_distance[key]), bins=10, color="b", ax=ax, vertical=True)
    ax.set_yticklabels('')
    if i!= len(cross_distance.keys())-1:
        ax.set_xticklabels('')
fig.text(0.01, 0.5, 'Cross-Shore Distance / m', va='center', rotation='vertical', fontsize=12)


KeyboardInterrupt: 

In [33]:
short_cross_distance = dict([])

#Select the transects to study (numbered from 0 to len(cross_distance))

transect_selection = [20, 70, 120, 170, 220, 250]

for i in range(len(transect_selection)):
    composite_array = [cross_distance['Transect_'+str(transect_selection[i-5])],cross_distance['Transect_'+str(transect_selection[i-4])],cross_distance['Transect_'+str(transect_selection[i-3])],cross_distance['Transect_'+str(transect_selection[i-2])],cross_distance['Transect_'+str(transect_selection[i-1])],cross_distance['Transect_'+str(transect_selection[i])],cross_distance['Transect_'+str(transect_selection[i+1])],cross_distance['Transect_'+str(transect_selection[i+2])],cross_distance['Transect_'+str(transect_selection[i+3])],cross_distance['Transect_'+str(transect_selection[i+4])],cross_distance['Transect_'+str(transect_selection[i+5])]]
    short_cross_distance[key] = np.mean(composite_array, axis=0)

In [57]:
fig = plt.figure(figsize=[15,10], tight_layout=True)
gs = gridspec.GridSpec(len(short_cross_distance),2, wspace=0.035, width_ratios=[3,1])
gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.2)
for i,key in enumerate(short_cross_distance.keys()):
    if np.all(np.isnan(short_cross_distance[key])):
        continue
    ax = fig.add_subplot(gs[i,0])
    ax.grid(linestyle=':', color='0.5')
    ax.set_ylim([-100,110])
    ax.plot(output_proj['dates'], short_cross_distance[key]- np.nanmedian(short_cross_distance[key]), 'o-', ms=6, mfc='w')
    #xticks = ax.get_xticks()
    #ax.set_xticks(xticks[::len(xticks) // 15]) # set new tick positions
    ax.tick_params(axis='x', rotation=90) # set tick rotation
    ax.xaxis.set_major_locator(MaxNLocator(20)) 
    #ax.set_ylabel('distance [m]', fontsize=12)
    ax.text(0.5,0.95, key, bbox=dict(boxstyle="square", ec='k',fc='w'), ha='center',va='top', transform=ax.transAxes, fontsize=14)
    if i!= len(short_cross_distance.keys())-1:
        ax.set_xticklabels('')
    ax = fig.add_subplot(gs[i,1])
    #ax.set_xlim([-50,50])
    ax.set_xlim([0,0.015])
    sns.distplot(short_cross_distance[key]- np.nanmedian(short_cross_distance[key]), bins=10, color="b", ax=ax, vertical=True)
    ax.set_yticklabels('')
    if i!= len(short_cross_distance.keys())-1:
        ax.set_xticklabels('')
fig.text(0.01, 0.5, 'Cross-Shore Distance / m', va='center', rotation='vertical', fontsize=12)


Text(0.01, 0.5, 'Cross-Shore Distance / m')

## Veg Line

In [5]:
BasePath = 'Data/' + sitename + '/Veglines'

if os.path.isdir(BasePath) is False:
    os.mkdir(BasePath)

settings = {
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': projection_epsg,     # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 50,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) for buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 100,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'bright',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    # add the inputs defined previously
    'inputs': inputs,
    'projection_epsg': 27700,
    'hausdorff_threshold':3500000000000
}

Digitize a reference vegetation line

In [8]:
%matplotlib qt
settings['reference_shoreline'] = Image_Processing.get_reference_sl(metadata, settings, polygon, dates)
settings['max_dist_ref'] = 100 # max distance (in meters) allowed from the reference shoreline

Extracts vegetation lines from all selected images. This'll take a while...

In [ ]:
%matplotlib qt
output, output_latlon, output_proj = VegetationLine.extract_veglines(metadata, settings, polygon, dates)

Mapping shorelines:
S2:   41%

In [8]:
#Alternatively, load the output file if previously processed
filepath = os.path.join(inputs['filepath'], sitename)
with open(os.path.join(filepath, sitename + '_output.pkl'), 'rb') as f:
    output = pickle.load(f)
with open(os.path.join(filepath, sitename + '_output_latlon.pkl'), 'rb') as f:
    output_latlon = pickle.load(f)
with open(os.path.join(filepath, sitename + '_output_proj.pkl'), 'rb') as f:
    output_proj = pickle.load(f)

In [9]:
output = Toolbox.remove_duplicates(output) # removes duplicates (images taken on the same date by the same satellite)
output_latlon = Toolbox.remove_duplicates(output_latlon)
output_proj = Toolbox.remove_duplicates(output_proj)

21 duplicates
21 duplicates
21 duplicates


In [13]:
#FN THAT REMOVES BY ACTIVE CHOICE GOES HERE

"""
b = LineString(output['shorelines'][j])
b

from ipywidgets import Checkbox, Output
import json

with open('data.txt', 'r') as f:
    data = json.load(f)

item = Checkbox(value=data['item'], description='item')
out = Output()

@out.capture()
def on_value_change(change):
    key = change['owner'].description
    value = change['new']
    
    print(f'Saving value: "{{ \'{key}\': {value} }}"')
    
    data[key] = value
    with open('data.txt', 'w') as f:
        json.dump(data, f)

item.observe(on_value_change, names='value')
display(item)
display(out)
"""

'\nb = LineString(output[\'shorelines\'][j])\nb\n\nfrom ipywidgets import Checkbox, Output\nimport json\n\nwith open(\'data.txt\', \'r\') as f:\n    data = json.load(f)\n\nitem = Checkbox(value=data[\'item\'], description=\'item\')\nout = Output()\n\n@out.capture()\ndef on_value_change(change):\n    key = change[\'owner\'].description\n    value = change[\'new\']\n    \n    print(f\'Saving value: "{{ \'{key}\': {value} }}"\')\n    \n    data[key] = value\n    with open(\'data.txt\', \'w\') as f:\n        json.dump(data, f)\n\nitem.observe(on_value_change, names=\'value\')\ndisplay(item)\ndisplay(out)\n'

Visual Display of Vegetation Lines. Will tank your PC if you've got lots of lines

In [ ]:
#Creates map object centred at ROI + adds compiled satellite image as base-layer
Map = geemap.Map(center=[polygon[0][0][1],polygon[0][0][0]],zoom=12)
Map.add_basemap('HYBRID')

#Generates colours for lines to be drawn in. Check out https://seaborn.pydata.org/tutorial/color_palettes.html for colour options...
palette = sns.color_palette("bright", len(output['shorelines']))
palette = palette.as_hex()

#Choose 'points' or 'lines' for the layer geometry
geomtype = 'points'

for i in range(len(output['shorelines'])-230):
    shore = dict([])
    print(output_latlon['shorelines'][i])
    if len(output_latlon['shorelines'][i])==0:
        continue
    shore = {'dates':[output_latlon['dates'][i]], 'shorelines':[output_latlon['shorelines'][i]], 'filename':[output_latlon['filename'][i]], 'cloud_cover':[output_latlon['cloud_cover'][i]], 'idx':[output_latlon['idx'][i]], 'Otsu_threshold':[output_latlon['Otsu_threshold'][i]], 'satname':[output_latlon['satname'][i]]}
    gdf = Toolbox.output_to_gdf(shore, geomtype)
    Line = geemap.geopandas_to_ee(gdf, geodesic=True)
    Map.addLayer(Line,{'color': str(palette[i])},'coast'+str(i))

Map

In [8]:
#Saves the veglines as individual shape-files locally under /Veglines.
direc = os.path.join(filepath, '/Veglines')
geomtype = 'lines'
name_prefix = 'Data/' + sitename + '/Veglines'

if os.path.isdir(direc) is False:
    os.mkdir(direc)

Toolbox.save_shapefiles(output_latlon, geomtype, name_prefix, sitename)

ref_line = np.delete(settings['reference_shoreline'],2,1)
ref = {'dates':['3000-12-30'], 'shorelines':[ref_line], 'filename':[0], 'cloud_cover':[0], 'geoaccuracy':[0], 'idx':[0], 'Otsu_threshold':[0], 'satname':[0]}
    
Toolbox.save_shapefiles(ref, geomtype, name_prefix, sitename)

In [9]:
#Produces Transects and Coast shape-files for the reference line

SmoothingWindowSize = 21
NoSmooths = 100
TransectSpacing = 10
DistanceInland = 350
DistanceOffshore = 350
BasePath = 'Data/' + sitename + '/Veglines'

Transects.produce_transects(SmoothingWindowSize, NoSmooths, TransectSpacing, DistanceInland, DistanceOffshore, image_epsg, sitename, BasePath + '/' + sitename + '_referenceLine')

#(Optional) Produces transects for all produced lines
#Transects.produce_transects_all(SmoothingWindowSize, NoSmooths, TransectSpacing, DistanceInland, DistanceOffshore, projection_epsg, BasePath)

Coast: Initialising Coast object
Coast.ReadCoastShp: Read Coastline, no of coast segments is 1
	Coastline    1 /    1
Coast: Smoothing CoastLines
Coast.WriteCoastShp: Writing coast line to a shapefile
Coast.GenerateTransectNormals: Generating CoastLine transects perpendicular to the coast
Coast.WriteTransectsShp: Writing coastal transects and attributes to a shapefile


In [10]:
#Defines all transects in a library.

TransectSpec =  '/' + sitename + '_referenceLine/Transect.shp'
geo = gpd.read_file(BasePath+TransectSpec)

transect_latlon, transect_proj = Transects.stuffIntoLibrary(geo, image_epsg, projection_epsg, filepath, sitename)

Current Progress: 99.77 %


In [10]:
#Or just load them if already produced
with open(os.path.join(filepath, sitename + '_transect_proj' + '.pkl'), 'rb') as f:
    transect_proj = pickle.load(f)
with open(os.path.join(filepath, sitename + '_transect_latlon' + '.pkl'), 'rb') as f:
    transect_latlon = pickle.load(f)

In [11]:
# defines the along-shore distance over which to consider vegetation line points to compute the median intersection (robust to outliers)
settings['along_dist'] = 50
cross_distance = Transects.compute_intersection(output_proj, transect_proj, settings, 'vegetation_') 

KeyboardInterrupt: 

In [11]:
#Or just load it if it already exists :)

cross_distance = dict([])

with open('Data/'+sitename+'/vegetation_transect_time_series.csv', newline='') as csvfile:
    spamreader = csv.DictReader(csvfile, delimiter=',', quotechar='|')
    for lines in spamreader:
        for i in range(len(lines)-2):
            cross_distance['Transect_'+str(i+1)] = []

with open('Data/'+sitename+'/vegetation_transect_time_series.csv', newline='') as csvfile:
    spamreader = csv.DictReader(csvfile, delimiter=',', quotechar='|')
    for lines in spamreader:
        for i in range(len(lines)-2):
            transect_name = 'Transect Transect_' + str(i+1)
            try:
                cross_distance['Transect_'+str(i+1)].append(float(lines[transect_name]))
            except:
                cross_distance['Transect_'+str(i+1)].append(np.nan)

## Analysis - Vegetation Edge

In [16]:
#Displays produced lines/transects

fig = plt.figure(figsize=[15,8], tight_layout=True)
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
#plt.xlim(509000,513000)
#plt.ylim(6244400,6247250)
plt.grid(linestyle=':', color='0.5')
for i in range(len(output_proj['shorelines'])):
    sl = output_proj['shorelines'][i]
    date = output_proj['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.')#, label=date.strptime('%d-%m-%Y'))
 
for i,key in enumerate(list(transect_proj.keys())):
    plt.plot(transect_proj[key][0,0],transect_proj[key][0,1], 'bo', ms=5)
    plt.plot(transect_proj[key][:,0],transect_proj[key][:,1],'k-',lw=1)
    #plt.text(transects_proj[key][0,0]-100, transects_proj[key][0,1]+100, key, va='center', ha='right', bbox=dict(boxstyle="square", ec='k',fc='w'))
plt.show()

In [19]:
#Displays the transects

for i,key in enumerate(list(transect_proj.keys())):
    plt.plot(transect_proj[key][0,0],transect_proj[key][0,1], 'bo', ms=5)
    plt.plot(transect_proj[key][:,0],transect_proj[key][:,1],'k-',lw=1)
    #plt.text(transects_proj[key][0,0]-100, transects_proj[key][0,1]+100, key, va='center', ha='right', bbox=dict(boxstyle="square", ec='k',fc='w'))
plt.show()

In [21]:
#Displays the lines

fig = plt.figure(figsize=[15,8])
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output_proj['shorelines'])):
    sl = output_proj['shorelines'][i]
    date = output_proj['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.')#, label=date.strftime('%d-%m-%Y'))
plt.legend()
plt.show()

No handles with labels found to put in legend.


In [22]:
#Cross-distance plots for ALL transects (do not bother if you are considering a LOT of transects)

fig = plt.figure(figsize=[15,12], tight_layout=True)
gs = gridspec.GridSpec(len(cross_distance),2, wspace=0.035, width_ratios=[3,1])
gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.2)
for i,key in enumerate(cross_distance.keys()):
    if np.all(np.isnan(cross_distance[key])):
        continue
    ax = fig.add_subplot(gs[i,0])
    ax.grid(linestyle=':', color='0.5')
    ax.set_ylim([-100,110])
    ax.plot(output['dates'], cross_distance[key]- np.nanmedian(cross_distance[key]), '-o', ms=6, mfc='w')
    #ax.set_ylabel('distance [m]', fontsize=12)
    ax.text(0.5,0.95, key, bbox=dict(boxstyle="square", ec='k',fc='w'), ha='center',va='top', transform=ax.transAxes, fontsize=14)
    if i!= len(cross_distance.keys())-1:
        ax.set_xticklabels('')
    ax = fig.add_subplot(gs[i,1])
    #ax.set_xlim([-50,50])
    ax.set_xlim([0,0.015])
    sns.distplot(cross_distance[key]- np.nanmedian(cross_distance[key]), bins=10, color="b", ax=ax, vertical=True)
    ax.set_yticklabels('')
    if i!= len(cross_distance.keys())-1:
        ax.set_xticklabels('')
fig.text(0.01, 0.5, 'Cross-Shore Distance / m', va='center', rotation='vertical', fontsize=12)


KeyboardInterrupt: 

In [91]:
#In this cell, you can iterate on transect range (we will use these ranges to analyse specific regions of the edge)

transect_range = [[0, 50],[51,110],[111,180],[181,240],[241,len(output['dates'])-1]] #enter 'full' to select all transects in one
#transect_colour = sns.color_palette("bright", len(transect_range))
transect_colour = ['#ff0000','#0084ff','#ff00f7','#00fa0c', '#ffb300', '#00ffcc','#7b00ff']

fig = plt.figure(figsize=[15,8], tight_layout=True)
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
#plt.xlim(509000,513000)
#plt.ylim(6244400,6247250)
plt.grid(linestyle=':', color='0.5')
for i in range(len(output_proj['shorelines'])):
    sl = output_proj['shorelines'][i]
    date = output_proj['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.')#, label=date.strptime('%d-%m-%Y'))

if transect_range == 'full':
    transect_range = [[0,len(transect_proj.keys())]]   

for i,key in enumerate(list(transect_proj.keys())):
    for j in range(len(transect_range)):
        if transect_range[j][0] <= i <= transect_range[j][1]:
            plt.plot(transect_proj[key][0,0],transect_proj[key][0,1], 'bo', ms=5,color=transect_colour[j])
            plt.plot(transect_proj[key][:,0],transect_proj[key][:,1],'k-',lw=1,color=transect_colour[j])
    #plt.text(transects_proj[key][0,0]-100, transects_proj[key][0,1]+100, key, va='center', ha='right', bbox=dict(boxstyle="square", ec='k',fc='w'))

plt.savefig('Data/' + sitename + '/jpg_files/transectsFull', bbox_inches='tight')
  
plt.show()

In [187]:
#Year by Year

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

fig, axs = plt.subplots(len(transect_range),sharex=True,figsize=(10, 12))
fig.text(0.005, 0.5, "Average Yearly Vegetation Cross-Edge Distance / m", va='center', rotation='vertical', fontsize=12)

for i in range(len(transect_range)):
    axs[i].set_title("Transects:"+str(transect_range[i][0])+"-"+str(transect_range[i][1]),backgroundcolor=transect_colour[i],color='white')
    if i != len(transect_range)-1:
        axs[i].xaxis.set_visible(False)
    if i == len(transect_range)-1:
        axs[i].set_xlabel("Year", fontsize=12)
    for j in range(transect_range[i][0],transect_range[i][1]):
        KEY = 'Transect_'+str(j+1)
        try:
            a, b, c, d, e = Toolbox.Separate_TimeSeries_year(cross_distance, output_proj, KEY)
            NaN_mask = np.isfinite(e)
            axs[i].plot(np.array(d)[NaN_mask],np.array(e)[NaN_mask])
        except:
            continue
            
plt.savefig('Data/' + sitename + '/jpg_files/avgYearlyVegPosition', bbox_inches='tight')


In [188]:
#Good at looking at seasonal patterns. Takes a while.

#plt.figure(figsize=[15,12])

months = ["Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", "Sept", "Oct", "Nov", "Dec"]
Month_dict = {"Jan":[], "Feb":[], "Mar":[], "Apr":[], "May":[], "June":[], "July":[], "Aug":[], "Sept":[], "Oct":[], "Nov":[], "Dec":[]}

Total_Month_Arr = []
test1 = []
test2 = []

fig, axs = plt.subplots(len(transect_range),sharex=True,figsize=(10, 12))

for l in range(len(transect_range)):

    for i in range(transect_range[l][0],transect_range[l][1]):
        KEY = 'Transect_'+str(i+1)
        try:
            a, b, c, d, e = Toolbox.Separate_TimeSeries_month(cross_distance, output_proj,KEY)

            zipped_lists = zip(d,e)
            s = sorted(zipped_lists)
            tuples = zip(s)

            new_d = []
            new_e = []

            sortedList = [list(tuple) for tuple in  tuples]

            for v in range(len(sortedList)):
                new_d.append(sortedList[v][0][0])
                new_e.append(sortedList[v][0][1])

            month_arr = []
            for j in range(len(d)):
                a = datetime.strptime(str(new_d[j]),'%m')
                month_arr.append(a.strftime('%b'))

            axs[l].scatter(month_arr,new_e,label=KEY)
            test1.append(new_d)
            test2.append(new_e)
        except:
            continue

    avg = []
    st_err = []
    Total_organised = []
    temp = []

    for k in range(len(test2[0])):
        for h in range(len(test2)):
            temp.append(test2[h][k])
        Total_organised.append(temp)
        avg.append(np.nanmean(temp))
        st_err.append(np.nanstd(temp)/(len(temp)**0.5))
        temp = []
    
    Total_Month_Arr.append(Total_organised)
    
    #plt.errorbar(month_arr,avg, yerr=st_err, color='k')
    axs[l].scatter(month_arr,avg, color='k', s=50, marker='x')

    #plt.legend()
    axs[l].set_title("Transects:"+str(transect_range[l][0])+"-"+str(transect_range[l][1]),backgroundcolor=transect_colour[l],color='white')

fig.text(0.01,0.5,"Averaged Monthly Vegetation Cross-Edge Distance / m", va='center', rotation='vertical')
plt.xlabel("Month")

plt.savefig('Data/' + sitename + '/jpg_files/monthScatter', bbox_inches='tight')


In [189]:
fig, axs = plt.subplots(len(transect_range),sharex=True,figsize=(10, 12))

for j in range(len(Total_Month_Arr)):
    
    axs[j].set_title("Transects:"+str(transect_range[j][0])+"-"+str(transect_range[j][1]),backgroundcolor=transect_colour[j],color='white')
    axs[j].boxplot(Total_Month_Arr[j],notch=True, flierprops = dict(marker='o', markersize=8, linestyle='none', markeredgecolor='r'))

fig.text(0.01,0.5,"Averaged Monthly Vegetation Cross-Edge Distance / m", va='center', rotation='vertical')
plt.xticks(new_d, month_arr)

plt.savefig('Data/' + sitename + '/jpg_files/monthBox', bbox_inches='tight')

plt.show()

In [215]:
def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])

    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value


def set_axis_style(ax, labels):
    ax.xaxis.set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels(labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    #ax.set_xlabel('Sample name')

fig, axs = plt.subplots(len(transect_range),sharex=True,figsize=(10, 12))

for j in range(len(Total_Month_Arr)):
    
    axs[j].set_title("Transects:"+str(transect_range[j][0])+"-"+str(transect_range[j][1]),backgroundcolor=transect_colour[j],color='white')
    parts = axs[j].violinplot(Total_Month_Arr[j], showmeans=False, showmedians=False, showextrema=False)

    for pc in parts['bodies']:
        pc.set_facecolor(transect_colour[j])
        pc.set_edgecolor('black')
        pc.set_alpha(0.7)

    quartile1, medians, quartile3 = np.percentile(Total_Month_Arr[j], [25, 50, 75], axis=1)
    whiskers = np.array([
        adjacent_values(sorted_array, q1, q3)
        for sorted_array, q1, q3 in zip(Total_Month_Arr[j], quartile1, quartile3)])
    whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 1]

    inds = np.arange(1, len(medians) + 1)
    axs[j].scatter(inds, medians, marker='o', color='white', s=30, zorder=3)
    axs[j].vlines(inds, quartile1, quartile3, color='k', linestyle='-', lw=5)
    axs[j].vlines(inds, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)
    
    if j == len(Total_Month_Arr):
        set_axis_style(axs[j], month_arr)

fig.text(0.005,0.5,"Averaged Monthly Vegetation Cross-Edge Distance / m", va='center', rotation='vertical')
plt.xticks(new_d, month_arr)

plt.savefig('Data/' + sitename + '/jpg_files/monthViolin', bbox_inches='tight')

plt.show()

In [201]:
#ignore for now

def sinusoidal_fit(x, A, B, C, D):
    return A* np.sin(B * x + C) + D

params, params_covariance = optimize.curve_fit(sinusoidal_fit, new_d, avg)

x = np.arange(0,max(new_d),0.01)

y = []
for i in range(len(x)):
    y.append(sinusoidal_fit(x[i], params[0], params[1], params[2], params[3]))
    
plt.errorbar(month_arr,avg, yerr=st_err, color='k')
plt.plot(x, y)

In [ ]:
#ignore for now

# Define plot space
fig, ax = plt.subplots(figsize=(10, 6))

# Define x and y axes
ax.plot(months, boulder_monthly_precip)


In [150]:
#ignore for now
plt.plot(months,Average_Month)

In [12]:
#array of colours for each of the averaged transect-analysis (add more if need be)
colours = ['#ff0000','#0084ff','#ff00f7','#00fa0c', '#ffb300', '#00ffcc','#7b00ff']

Rows = []

with open('Data/'+sitename+'/vegetation_transect_time_series.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        Rows.append(row[2:])

cross_distance_condensed, standard_err_condensed, transect_condensed, Dates = Transects.transect_compiler(Rows, transect_proj, transect_interval, output)

In [113]:
#Package this into Toolbox

#This averages over every (transect_interval) transects.

colours = ['#ff0000','#0084ff','#ff00f7','#00fa0c', '#ffb300', '#00ffcc','#7b00ff']

#transect_range

Rows = []

with open('Data/'+sitename+'/vegetation_transect_time_series.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        Rows.append(row[2:])
        
cross_distance_condensed = dict([])
standard_err_condensed = dict([])
transect_condensed = dict([])
Dates = dict([])

for i in range(len(transect_range)):
    cross_arr = []
    trans_arr = []
    for j in range(transect_range[i][0],transect_range[i][1]):
        try:
            arr = []
            for k in range(len(Rows)-1):
                try:
                    arr.append(float(Rows[k][j]))
                except:
                    arr.append(np.nan)
            cross_arr.append(arr)
            trans_arr.append(transect_proj[list(transect_proj.keys())[j]])
        except:
            continue
    std = np.nanstd(cross_arr,0)
    for j in range(len(std)):
        std[j] = std[j]/transect_interval**0.5
    
    NaN_mask = np.isfinite(np.nanmean(cross_arr,0))
    cross_distance_condensed['Transect_'+str(transect_range[i][0])+'-'+str(transect_range[i][1])] = np.nanmean(cross_arr,0).astype(np.double)[NaN_mask]
    standard_err_condensed['Transect_'+str(transect_range[i][0])+'-'+str(transect_range[i][1])] = std.astype(np.double)[NaN_mask]
    Dates['Transect_'+str(transect_range[i][0])+'-'+str(transect_range[i][1])] = np.array(output['dates'])[NaN_mask]
    transect_condensed['Transect_'+str(transect_range[i][0])+'-'+str(transect_range[i][1])] = np.mean(trans_arr,0).astype(np.double)#[NaN_mask]

"\nfor i in range(len(Rows[1])):\n    \n    arr = []\n    for k in range(len(Rows)):\n        if k == 0:\n            continue\n        try:\n            arr.append(float(Rows[k][i]))\n        except:\n            arr.append(np.nan)\n\n    cross_arr.append(arr)\n    \n    trans_arr.append(transect_proj[list(transect_proj.keys())[i]])\n\n    if (i%transect_interval != 0) or i==0:\n        continue\n        \n    std = np.nanstd(cross_arr,0)\n    for j in range(len(std)):\n        std[j] = std[j]/transect_interval**0.5\n    \n    NaN_mask = np.isfinite(np.nanmean(cross_arr,0))\n    cross_distance_condensed['Transect_'+str(new_Transect)+'-'+str(new_Transect+transect_interval-1)] = np.nanmean(cross_arr,0).astype(np.double)[NaN_mask]\n    standard_err_condensed['Transect_'+str(new_Transect)+'-'+str(new_Transect+transect_interval-1)] = std.astype(np.double)[NaN_mask]\n    Dates['Transect_'+str(new_Transect)+'-'+str(new_Transect+transect_interval-1)] = np.array(output['dates'])[NaN_mask]\n   

In [186]:
fig = plt.figure(figsize=[15,12], tight_layout=True)
gs = gridspec.GridSpec(len(cross_distance_condensed),2, wspace=0.035, width_ratios=[4,1])
gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.05)

x = np.arange(datetime(1984,1,1), datetime(2022,1,1), timedelta(days=100)).astype(str)
y = [0]*139

for i,key in enumerate(cross_distance_condensed.keys()):
    
    if np.all(np.isnan(cross_distance_condensed[key])):
        continue
        
    ax = fig.add_subplot(gs[i,0])
    ax.grid(linestyle=':', color='0.5')
    ax.set_ylim([min(cross_distance_condensed[key]- np.nanmedian(cross_distance_condensed[key]))-5,max(cross_distance_condensed[key]- np.nanmedian(cross_distance_condensed[key]))+5])
    dates = matplotlib.dates.date2num(Dates[key])
    ax.errorbar(dates, cross_distance_condensed[key]- np.nanmedian(cross_distance_condensed[key]), yerr = standard_err_condensed[key],fmt='-o',ecolor= 'k', color= colours[i], ms=6, mfc='w')
    #ax.plot_date(Dates[key], cross_distance_condensed[key]- np.nanmedian(cross_distance_condensed[key]), xdate=True,fmt='-o', color= colours[i], ms=6, mfc='w')
    #ax.fill_between(Dates[key], 0, cross_distance_condensed[key]- np.nanmedian(cross_distance_condensed[key]),alpha=0.5,color=colours[i])
    
    #ax.plot(x,y,color='k')
    
    #ax.text(0.5,0.95, key, bbox=dict(boxstyle="square", ec='k',fc='w'), ha='center',va='top', transform=ax.transAxes, fontsize=14)
    #ax.set_xlim(min(Dates[key]),max(Dates[key]))
    #ax.set_xticklabels(linspace_datetime64(np.datetime64(min(Dates[key])),np.datetime64(max(Dates[key])), 7))#,rotation=45)
    #ax.tick_params(axis='both', which='minor', labelsize=12)
    #ax.grid(False,axis='x')
    
    ax.fill_between(dates, 0, cross_distance_condensed[key]- np.nanmedian(cross_distance_condensed[key]),alpha=0.5,color=colours[i])
    ax.set_title("Transects:"+str(transect_range[i][0])+"-"+str(transect_range[i][1]),backgroundcolor=transect_colour[i],color='white')
    #ax.xaxis.set_major_locator(locator)
    #ax.set_xticks(dates, minor=False)
    ax.set_xticklabels(['1982','1986','1992','1998','2004','2010','2016','2020','2014','2018','2022'])
    #ax.set_xticks(dates, minor=False)
    #ax.set_xticklabels(Dates[key])
    #locator=MaxNLocator(12)
    #ax.xaxis.set_major_locator(locator)
    if i!= len(cross_distance_condensed.keys())-1:
        ax.set_xticklabels('')
    #ax.set_xticks(Dates[key])

    ax = fig.add_subplot(gs[i,1])
    #ax.set_xlim([-50,50])
    ax.set_xlim([0,0.020])
    sns.distplot(cross_distance_condensed[key]- np.nanmedian(cross_distance_condensed[key]), bins=10, color=colours[i], ax=ax, vertical=True)
    ax.set_yticklabels('')
    
    if i!= len(cross_distance_condensed.keys())-1:
        ax.set_xticklabels('')
        ax.set_xlabel('')
        
fig.text(0.01, 0.5, 'Cross Vegetation-Edge Distance / m', va='center', rotation='vertical', fontsize=13.8)

plt.savefig('Data/' + sitename + '/jpg_files/crossEdgeDistances', bbox_inches='tight')

In [225]:
ref_sl_conv = Toolbox.convert_epsg(settings['reference_shoreline'], 32630, 27700)[:,:-1]

vv = dict([])
vv['1'] = [ref_sl_conv]

#Displays produced lines/transects

fig = plt.figure()#figsize=[15,8], tight_layout=True)
plt.axis('equal')
#plt.xlabel('Eastings')
#plt.ylabel('Northings')
plt.xlim(min(vv['1'][0][:,0]),max(vv['1'][0][:,0]))
plt.xticks('')
plt.yticks('')
plt.ylim(min(vv['1'][0][:,1])-50,max(vv['1'][0][:,1])+50)
plt.grid(linestyle=':', color='0.5')
for i in range(len(vv['1'])):
    sl = vv['1'][i]
    date = vv['1'][i]
    plt.plot(sl[:,0], sl[:,1], '.', color='k')#, label=date.strptime('%d-%m-%Y'))
 
for i,key in enumerate(list(transect_condensed.keys())):
    plt.plot(transect_condensed[key][0,0],transect_condensed[key][0,1], 'bo', color= colours[i], ms=5)
    plt.plot(transect_condensed[key][:,0],transect_condensed[key][:,1],'k-', color= colours[i], lw=1)
    plt.text(transect_condensed[key][1][0],transect_condensed[key][1][1], key, va='bottom', ha='right', bbox=dict(boxstyle="round", ec='k',fc='w'), fontsize=10)

plt.savefig('Data/' + sitename + '/jpg_files/refEdge_Transects', bbox_inches='tight')
plt.show()

In [ ]:
i=18
produced = False
while produced is False:
    try:
        print(i)
        im_ms, georef, cloud_mask, im_extra, im_QA, im_nodata = Image_Processing.preprocess_single(i, 'S2', settings, polygon, dates)
        produced = True
        break
    except:
        i+=1
        continue
        
im_RGB = Image_Processing.rescale_image_intensity(im_ms[:,:,[2,1,0]], cloud_mask, 99.9)
fig = plt.figure()
fig.set_size_inches([18,9])
fig.set_tight_layout(True)
ax1 = fig.add_subplot(111)
ax1.axis('off')
ax1.imshow(im_RGB)
plt.rcParams['savefig.jpeg_quality'] = 100

fig.savefig(os.path.join('Data/' + sitename + '/jpg_files/SatelliteImg.jpg'), dpi=150)

18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
28

In [251]:
def format_axes(fig):
    for i, ax in enumerate(fig.axes):
        ax.text(0.5, 0.5, "ax%d" % (i+1), va="center", ha="center")
        ax.tick_params(labelbottom=False, labelleft=False)

fig = plt.figure(constrained_layout=True, figsize=(15,12))

gs = gridspec.GridSpec(6, 3, figure=fig, wspace=0.00001, hspace=0.1)

ax1 = fig.add_subplot(gs[0,0])
# identical to ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=3))
ax2 = fig.add_subplot(gs[0,1])
ax3 = fig.add_subplot(gs[:2,2])
ax4 = fig.add_subplot(gs[1, :2])
#cross-dist
cross1 = fig.add_subplot(gs[2, :])
cross2 = fig.add_subplot(gs[3, :])
cross3 = fig.add_subplot(gs[4, :])
cross4 = fig.add_subplot(gs[5, :])

fig.suptitle("GridSpec")
format_axes(fig)



plt.show()